In [1]:
using Plots
#using PlotlyJS
using Random, Distributions, StatsBase, LinearAlgebra, DelimitedFiles, Optim
Random.seed!()

TaskLocalRNG()

In [2]:
# get observed data and known covariates
io = open("../data/input/X.txt","r")
X = readdlm(io, Float64)
close(io)
#X = X[:,50:750]

io = open("../data/input/Y.txt","r")
Y = readdlm(io, Float64)
close(io)
#Y = Y[50:750,50:750]

N = length(X[1,:])
P = length(X[:,1])
K = 4     # I know that the data was generate with K = 4. In principle one should do model selection to discover it

4

In [4]:
# approximate ELBO
function ELBO(ϕ::Array{Float64, 3}, λ::Array{Float64, 2}, ν::Vector{Matrix{Float64}},
    Σ::Array{Float64, 2}, σ_2::Array{Float64, 1}, B::Array{Float64, 2}, ρ::Float64, μ::Array{Float64, 2})
    apprELBO = 0.0
    inv_Σ = inv(Σ)
    apprELBO = 0.5 * N * log(det(inv_Σ))
    for i in 1:N
        apprELBO -= 0.5 * (dot(λ[i,:] .- μ[:,i], inv_Σ, λ[i,:] .- μ[:,i]) + tr(inv_Σ*ν[i])) #first 2 terms
    end
    
    for i in 1:N
        for j in 1:N
            if i != j
                apprELBO += dot(ϕ[i,j,:],λ[i,:])# vcat(λ[i,:], 1.0)) #third term
                for k in 1:K
                    apprELBO -= ϕ[i,j,k]*log(ϕ[i,j,k]) #last entropic term
                end
            end
        end
    end
    
    for i in 1:N
        theta = zeros(K)
        theta .= exp.(λ[i,:])  #exp.(vcat(λ[i,:], 1.0))
        theta /= sum(theta)
        # second line of the expression above
        apprELBO -= (N-1) * ( (log(sum(exp.(λ[i,:])))) + 0.5*tr((diagm(theta) .- theta * theta')*ν[i]) )
        #gaussian entropic term
        apprELBO += 0.5*log(det(ν[i]))
    end
    
    #likelihood term
    for i in 1:N
        for j in 1:i-1
            for k in 1:K
                for g in 1:K
                    #logP = Y[i,j] * ( log(B[k,g]*(1-ρ)) - abs(i-j)/N) - B[k,g]*(1-ρ)*exp(-abs(i-j)/N)
                    logP = Y[i,j]*log(B[k,g]*(1-ρ)) + (1-Y[i,j])*log(1-B[k,g]*(1-ρ))
                    apprELBO += ϕ[i,j,k]*ϕ[j,i,g]*logP
                end
            end
        end
    end
    return apprELBO
end

ELBO (generic function with 1 method)

In [9]:
# define the function to optimize
function f(η_i::Array{Float64, 1}, ϕ_i::Array{Float64, 1}, inv_Σ::Array{Float64, 2}, μ_i::Array{Float64},N::Int)
    f = 0.5 * dot(η_i .- μ_i, inv_Σ, η_i .- μ_i) - dot(η_i, ϕ_i) +(N-1)*log(sum(exp.(η_i)))
    return f
end


function gradf!(G, η_i::Array{Float64, 1}, ϕ_i::Array{Float64, 1}, inv_Σ::Array{Float64, 2}, μ_i::Array{Float64},N::Int)
    G .= exp.(η_i)/sum(exp.(η_i))*(N-1) .- ϕ_i .+ inv_Σ*(η_i .- μ_i) 
    #G .= -exp.(η_i)/sum(exp.(η_i)) .+ ϕ_i/(N-1) - inv_Σ*(η_i .- μ_i)/(N-1)
end

function hessf!(H, η_i::Array{Float64, 1}, inv_Σ::Array{Float64, 2}, μ_i::Array{Float64},N::Int)
    theta = exp.(η_i)/sum(exp.(η_i))
    H .= + (N-1)*(diagm(theta) .- theta*theta') .+ inv_Σ
    #H .= - (diagm(theta) .- theta*theta') .- inv_Σ/(N-1)
end

hessf! (generic function with 1 method)

In [10]:
# variational parameters for the E-step
ϕ = ones(N,N,K) #.* 1/K  # initialized as uniform distributions
for i in 1:N
    for j in 1:N
        ϕ[i,j,:] = rand(Dirichlet(K,1.0))
    end
end
λ = randn(N,K)    # mean vectors for the gaussians for every node
ν = [Matrix(1.0I, K, K) for i in 1:N] # covariance matrices for the gaussians. This is a vector of matrices

# parameters to be optimized in the M-step
#Σ = Matrix(1.0I, K, K)    # global covariance matrix
Σ = rand(Wishart(K,Matrix(.5I,K, K)))
σ_2 = rand(Gamma(1,1), K);      # prior covariance on the transformation coefficients Γ

B = zeros(K,K)
for k in 1:K
    for g in 1:k
        B[k,g] = rand()*0.04
        B[g,k] = B[k,g]
    end
end
B .+= Matrix(1.0I, K, K)*0.8
ρ = 0.1

Γ = zeros(K,P)
for k in 1:K
    Γ[k,:] .= randn(P)* sqrt(σ_2[k])
end

μ = Γ * X;
for i in 1:N
    ϕ[i,i,:] .= 0
end

In [21]:
for m in 1:1
    for i in 1:1
        inv_Σ = inv(Σ)
        G = zeros(K)
        H = zeros(K,K)
        ϕ_i = sum(ϕ[i,:,:],dims=1)[1,:]
        μ_i = μ[:,i]
        res = optimize(η_i -> f(η_i, ϕ_i, inv_Σ, μ_i, N), (G, η_i) -> gradf!(G,η_i, ϕ_i, inv_Σ, μ_i, N), randn(K), BFGS())
        #res = optimize(η_i -> f(η_i, ϕ_i, inv_Σ, μ_i, N), randn(K))
        println(res)
        #println(minimum(res))
        η_i = Optim.minimizer(res)
        hessf!(H, η_i, inv_Σ, μ_i, N)
        #println(inv(H))
        println(Optim.minimum(res))
    end
end

 * Status: success

 * Candidate solution
    Final objective value:     1.386509e+03

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 6.74e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 7.85e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.55e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.28e-16 ≰ 0.0e+00
    |g(x)|                 = 3.86e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    30
    ∇f(x) calls:   30

1386.50885964289


In [ ]:
* Status: success

 * Candidate solution
    Final objective value:     1.386509e+03

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    152
    f(x) calls:    273

1386.5088596457194